# Scaling inventory on footprints 

This example shows how we can retrieve how much emissions are being seen at 
a measurement location, based on geospatial footprint of that site.

In [ ]:
from pathlib import Path
import numpy as np
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
from shapely.creation import polygons
from emiproc.grids import RegularGrid, GeoPandasGrid
from emiproc.regrid import remap_inventory
from emiproc.inventories.zurich import MapLuftZurich
from emiproc.inventories.utils import group_categories



footprints_dir = Path('footprints')

footprint_file = footprints_dir / 'zurich_footprint_raster_fclim2d_2303071400.txt'
x_file = footprints_dir / 'zurich_footprint_raster_x2d2303071400.txt'
y_file = footprints_dir / 'zurich_footprint_raster_y2d2303071400.txt'

measurement_coordinates = 2680911.322, 1248390.798

In [ ]:


footprint = np.loadtxt(footprint_file)
x = np.loadtxt(x_file)
y = np.loadtxt(y_file)


In [ ]:
# Get only the cells of the grid where footprint is greater than 0
mask = footprint > 1e-20
x_coords = x[mask] + measurement_coordinates[0]
y_coords = y[mask] + measurement_coordinates[1]
d2 = 5.
coords = np.array(
    [
        [x, y]
        for x, y in zip(
            [x_coords -d2, x_coords - d2, x_coords + d2, x_coords + d2],
            [y_coords - d2, y_coords + d2, y_coords + d2, y_coords - d2],
        )
    ]
)
coords = np.rollaxis(coords, -1, 0)
# Create a gdf with the mask
gdf = gpd.GeoDataFrame(
    {
        'footprint': footprint[mask],
    },
    geometry=polygons(coords),
    crs="LV95"
)
grid = GeoPandasGrid(gdf)
gdf

In [ ]:
file = Path("mapLuft_2020_v2021.gdb")

zh_inv = MapLuftZurich(file, ['CO2'])

# Group the categories 
ZH_2_GNFR_CO2 = {
    # PublicPower
    "GNFR_A": [
        "c2201_BHKW_Emissionen_Kanton", # ???
        "c2301_KHKWKehricht_Emissionen_Kanton",
        "c2302_KHKWErdgas_Emissionen_Kanton",
        "c2303_KHKWHeizoel_Emissionen_Kanton",
    ],
    # Industry
    "GNFR_B": [
        "c3201_Notstromanlagen_Emissionen_Kanton",
        "c3301_Prozessenergie_Emissionen_Kanton",
        "c3410_Bierbrauereien_Emissionen_Kanton",
        "c3411_Brotproduktion_Emissionen_Kanton",
    ],
    # Other stationary combustion (services, residential, agriculture)
    "GNFR_C": [
        "c2101_Oelheizungen_Emissionen_Kanton",
        "c2102_Gasheizungen_Emissionen_Kanton",
        "c2103_HolzheizungenLokalisiert_Emissionen_Kanton",
        "c2104_HolzheizungenDispers_Emissionen_Kanton",
        "c2105_Warmwassererzeuger_Emissionen_Kanton",
    ],
    # Fugitives
    "GNFR_D": [
        "c3416_Tankstellen_Emissionen_Kanton",
    ],
    # Solvents and product use
    #"GNFR_E": [
    #],
    # Road transport
    "GNFR_F": [
        "c1301_Personenwagen_Emissionen_Kanton",
        "c1302_Lastwagen_Emissionen_Kanton",
        "c1303_Motorraeder_Emissionen_Kanton",
        "c1304_Linienbusse_Emissionen_Kanton",
        "c1306_StartStopTankatmung_Emissionen_Kanton",
        "c1307_Lieferwagen_Emissionen_Kanton",
        "c1308_Reisebusse_Emissionen_Kanton",
    ],
    # Shipping
    "GNFR_G": [
        "c1101_Linienschiffe_Emissionen_Kanton",
        "c1102_PrivaterBootsverkehr_Emissionen_Kanton",
    ],
    # Offroad mobility
    "GNFR_I": [
        # c31xx are construction stuff
        "c3101_MaschinenHochbau_Emissionen_Kanton",
        "c3104_MaschinenTiefbau_Emissionen_Kanton",
        "c3419_IndustrielleFZ_Emissionen_Kanton",
        "c4101_ForstwirtschaftlicheFZ_Emissionen_Kanton",
        "c4201_LandwirtschaftlicheFZ_Emissionen_Kanton",
    ],
    # Waste
    "GNFR_J": [
        "c2401_Klaerschlammverwertung_Emissionen_Kanton",
        "c3418_Vergaerwerk_Emissionen_Kanton",
        "c3414_Krematorium_Emissionen_Kanton",
        "c5201_Gruenabfallverbrennung_Emissionen_Kanton",
        "c5301_HolzoefenKleingarten_Emissionen_Kanton",
        "c5401_AbfallverbrennungHaus_Emissionen_Kanton",
    ],
    # AgriLivestock
    # AgriOther
    #"GNFR_L": [
    #],
    # Others
    "GNFR_R": [
        "c5601_Feuerwerke_Emissionen_Kanton",
        "c5701_Tabakwaren_Emissionen_Kanton",
        "c5801_BrandFeuerschaeden_Emissionen_Kanton",
    ],
}

zh_groupped = group_categories(zh_inv, ZH_2_GNFR_CO2)


In [ ]:

regrided = remap_inventory(zh_groupped, grid)

In [ ]:
regrided.total_emissions.plot.bar()